In [ ]:
import numpy as np
import pandas as pd
import cv2
import h5py

In [ ]:
def read_many_hdf5(name):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Open the HDF5 file
    file = h5py.File(f"{name}.h5", "r+")

    images = np.array(file["/images"])#.astype("uint8")
    labels = np.array(file["/meta"])#.astype("uint8")
    return images, labels


In [ ]:
X,y =  read_many_hdf5('aadhaar_images')

In [ ]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import backend as K

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer='l2'))
model.add(layers.Dense(1, activation='sigmoid',kernel_initializer='he_normal'))

In [ ]:
conv_base.trainable = False

In [ ]:
model.summary()

In [ ]:

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=[f1])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator( rescale=1./255,   
        zoom_range = 0.05,  
        width_shift_range=0.2,  
        height_shift_range=0.2,  
        horizontal_flip=False,  
        vertical_flip=False,
        shear_range=0.05
        
                        )

In [ ]:
model.fit(datagen.flow(X,y,shuffle=False),epochs=2)

In [ ]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    img = frame
    
    frame = cv2.resize(frame,(224,224),interpolation=cv2.INTER_AREA)
    frame = frame /255
    frame = frame.reshape(1,224,224,3)
    output = model.predict_classes(frame)
    if output == [[1]]:
        img = cv2.putText(img,'Found', (30,30) , cv2.FONT_HERSHEY_COMPLEX , 1  , (0,255,0) , 2)
        cv2.imshow('webcam',img)
        count+=1
        
    if cv2.waitKey(1) == 13:
        break
    cv2.putText(img,'No Aadhaar Card Found', (30,30) , cv2.FONT_HERSHEY_COMPLEX , 1 , (255,0,0) , 2)
    cv2.imshow('webcam',img)
cap.release()
cv2.destroyAllWindows()